In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ****Import Library 

In [ ]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
import matplotlib .pyplot as plt 
import seaborn as sns
from sklearn .preprocessing import StandardScaler
from scipy.stats import norm
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer


# ****Read Data 

In [ ]:
data = pd.read_csv('../input/used-car-dataset-ford-and-mercedes/audi.csv')

In [ ]:
print(data.shape)
data .head()


In [ ]:
columns = data.columns 
columns 

In [ ]:
## Knowning some statistical information
data.describe()

In [ ]:
data.info


In [ ]:
## checking for null values.
print(data.isnull().sum())

# ****Convert  the column that containing string data to numerical number

In [ ]:
## convert the "Model" column
le = LabelEncoder()
le.fit(data['model'])
le.transform(data['model'])
data['model'] = le.transform(data['model'])

In [ ]:
## convert the "transmission" column
le = LabelEncoder()
le.fit(data['transmission'])
le.transform(data['transmission'])
data['transmission'] = le.transform(data['transmission'])

In [ ]:
# ## convert the "fuelType" column
le = LabelEncoder()
le.fit(data['fuelType'])
le.transform(data['fuelType'])
data['fuelType'] = le.transform(data['fuelType'])

In [ ]:
print(data.shape)
data.head()

# ****Spliting Data into Train_data and Test_Data

In [ ]:
train_data = data.iloc[:10000,:]
test_data = data.iloc[10000:,:]
print(train_data.shape)
print(test_data.shape)

In [ ]:
test_data = test_data.drop(columns = 'price')
test_data.shape

# ****Split data to X and y

In [ ]:

y = train_data['price']
X = train_data.drop(columns = 'price')
print(X.shape)
print(y.shape)

# ****Draw the output and Draw the relationShip between input and output

In [ ]:
#histogram and normal probability plot
sns.distplot(y, fit=norm);
fig = plt.figure()
res = stats.probplot(y, plot=plt)


In [ ]:
y  = np.log(y)


In [ ]:
##transformed histogram and normal probability plot
sns.distplot(y, fit=norm);
fig = plt.figure()
res = stats.probplot(y, plot=plt)

In [ ]:
y = train_data['price']
y

In [ ]:
numerical_features = data.columns
fig = plt.figure(figsize=(25,40)) #figure size
o = 4  # number of rows
q = 3  # number of columns
w = 1  # initialize plot counter

for feat in numerical_features:
    plt.subplot(o, q, w)
    sns.kdeplot(x=data[feat])
#     plt.hist(x=data[feat])
#     plt.scatter(z = data[feat])
    w+=1
    
plt.tight_layout()
plt.show()

In [ ]:
## Draw the releationship between output and input

model = X['model']
year = X['year']
trans = X['transmission']
mile = X['mileage']
fuel = X['fuelType']
tax = X['tax']
mpg = X['mpg']
eng = X['engineSize']

##.........................................../


fig, ax = plt.subplots(2,4,figsize=(30,20))
ax[0,0].scatter(model,y)
ax[0,1].scatter(year,y)
ax[0,2].scatter(trans,y)
ax[0,3].scatter(mile,y)
ax[1,0].scatter(fuel,y)
ax[1,1].scatter(tax,y)
ax[1,2].scatter(mpg,y)
ax[1,3].scatter(eng,y)


# ****Scaling data 

In [ ]:
## Scaling Model with standar scalar 

scaler = StandardScaler()
X = scaler.fit_transform(X)
print(X.shape)



# ****Spliting data to X_train, y_train,X_test and y_test

In [ ]:
## spliting data into X_train , y_train,X_test,y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=44, shuffle =True)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# ****apply the RandomForestRegressor Algorithm

In [ ]:
RandomForestRegressorModel = RandomForestRegressor(n_estimators=50,max_depth=15, random_state=33,min_samples_split=2,min_samples_leaf=2
                                                ,n_jobs=-1 )
                                                  
RandomForestRegressorModel.fit(X_train, y_train)

In [ ]:
#Calculating Details
print('Random Forest Regressor Train Score is : ' , RandomForestRegressorModel.score(X_train, y_train))
print('Random Forest Regressor Test Score is : ' , RandomForestRegressorModel.score(X_test, y_test))

In [ ]:
#Calculating Prediction
y_pred =  RandomForestRegressorModel.predict(X_test)

print('Predicted Value for RandomForestRegressorModelis : ' , y_pred)
print( ' y : ' , y[:10])

print(y_pred.shape)


In [ ]:
y_pred= pd.DataFrame(y_pred,columns=['Expected'])
y_pred

In [ ]:
y= pd.DataFrame(y)
y

In [ ]:
## comparison between the exppected Value "y_pred " and the actual Values "y"
y['Expected'] = y_pred
y.head(10)


In [ ]:
MAEValue = mean_absolute_error(y_test, y_pred, multioutput='uniform_average')
print("MAEValue is :" ,MAEValue)

In [ ]:
MSEValue= mean_squared_error(y_test, y_pred, multioutput='raw_values')
print("MSEValue is : " , MSEValue)


# ****knowing the best features

In [ ]:
##remove the missing value data
ImputedModule = SimpleImputer(missing_values = np.nan, strategy ='mean')
Imputedy = ImputedModule.fit(y)
y = Imputedy.transform(y)
y


In [ ]:
## Knowing the best features 
sel = SelectFromModel(RandomForestRegressor( )) 
sel.fit(X,y)
selected_features = sel.transform(X)
sel.get_support()

In [ ]:
feature_importance =  RandomForestRegressorModel.feature_importances_
feature_importance


In [ ]:
feature_importance= pd.DataFrame(feature_importance,columns=['feature'])
feature_importance

# ****Calculate Prediction

In [ ]:
test_preds = RandomForestRegressorModel.predict(test_data)


In [ ]:
test_preds= pd.DataFrame(test_preds,columns = ['predic in test'])
test_preds

In [ ]:
## comparison between the Prediction Value in trainData and the Prediction Value in testData
test_preds['predic in train'] = y_pred
test_preds 

In [ ]:
y = train_data['price']
y= pd.DataFrame(y)
y


In [ ]:
## comparison between the Prediction Value in trainData , the Prediction Value in testData and the Actual Value
test_preds['price'] = y
test_preds 